# Music migrations map
This script creates geospatial data for Cisco Bradley's Global Jazz Diasporas project at the Pratt Institute Music & Migration Lab. It ingests the historical database of musician migrations, geocodes the migrations, and calculates metrics about the migration network. 

### Set up for replication
This script was designed to be run in ArcGIS Pro 3.4 using Python 3. To replicate, create a new ArcGIS project, add this notebook, and then run as follows. 

In [1]:
import arcpy
import os
import pandas

In [2]:
#set up workspaces and check database
aprx = arcpy.mp.ArcGISProject("Current")
default_gdb = aprx.defaultGeodatabase
default_folder = aprx.homeFolder
arcpy.env.overwriteOutput = True
print("Directory: " + default_folder)
print("Geodatabase: " + default_gdb)

Directory: C:\Users\johnl\My Drive (jlauerma@pratt.edu)\Research\Music and Migration
Geodatabase: C:\Users\johnl\My Drive (jlauerma@pratt.edu)\Research\Music and Migration\MusicMigration.gdb


### Load and edit tabular data
The source data is stored as a Google Sheet to enable ongoing updates by the research team. The script pulls a CSV from the Google Sheets API and then loads to a pandas data frame. It then processes the data for mapping by creating a unique identification key for each musician and each migration segment. Genre details are also defined and split into the wide data structure needed for subsequent mapping. 

In [3]:
# load the data from Google Sheets
file_url = "https://docs.google.com/spreadsheets/d/e/2PACX-1vTocsXDPTgZKf9P7JkGUd-pg0Hj5huEShxLSkIyLArLaZxUFmqu3xYd33o9Oj1FJQvnl2IlMGGgAv5Y/pub?gid=0&single=true&output=csv"
data = pandas.read_csv(file_url)
data.columns

Index(['Arrival Country', 'Arrival State', 'Arrival County', 'Arrival City',
       'Musician Name', 'Arrival', 'Depart', 'Depart Country', 'Depart State',
       'Depart County', 'Depart City', 'Final Destination', 'Genre',
       'Instrument', 'Free', 'Roots', 'Dyn', 'Public', 'Dynasty Name',
       'Unnamed: 19', 'Unnamed: 20', 'Unnamed: 21'],
      dtype='object')

In [4]:
# drop unnessary columns
data = data.drop(columns = ['Unnamed: 19', 'Unnamed: 20', 'Unnamed: 21'])
data.columns

Index(['Arrival Country', 'Arrival State', 'Arrival County', 'Arrival City',
       'Musician Name', 'Arrival', 'Depart', 'Depart Country', 'Depart State',
       'Depart County', 'Depart City', 'Final Destination', 'Genre',
       'Instrument', 'Free', 'Roots', 'Dyn', 'Public', 'Dynasty Name'],
      dtype='object')

In [5]:
#create a unique ID for each musician 
data = data.sort_values(by='Musician Name')
data['MusicianID'] = data['Musician Name'].factorize()[0]+1
MusicianIDs = data['MusicianID'].nunique()
print(MusicianIDs, "musicians listed")

1505 musicians listed


In [6]:
#create a unique ID for each segement on the network
data['SegmentID'] = data['MusicianID'].astype(str) + data['Arrival'].astype(str) + data['Depart'].astype(str)
segmentIDs = data['SegmentID'].nunique()
print(segmentIDs, "unique travel paths listed")

7466 unique travel paths listed


In [7]:
#classify genre information
## dictionary mapping abbreviations to genres
genre_mapping = {
    'AC': 'Afro-Cuban Jazz',
    'AF': 'African Jazz',
    'AJ': 'Acid Jazz',
    'AR': 'Acid Rock',
    'BB': 'Bebop',
    'BG': 'Boogaloo',
    'BL': 'Blues',
    'BN': 'Bossa Nova',
    'BR': 'Broadway',
    'BS': 'Brass Band',
    'BW': 'Boogie-Woogie',
    'CH': 'Spirituals',
    'CI': 'Circus',
    'CL': 'European Classical',
    'CV': 'Folk Cape Verde',
    'CY': 'Country',
    'DI': 'Disco',
    'DW': 'Doo-Wop',
    'EJ': 'Early Jazz',
    'EL': 'Easy Listening',
    'FJ': 'Free Jazz',
    'FK': 'Funk',
    'FO': 'Folk American',
    'FU': 'Fusion',
    'GJ': 'General Jazz',
    'GO': 'Gospel',
    'HB': 'Hard Bop',
    'HH': 'Hip Hop',
    'HO': 'House',
    'JB': 'Jump Blues',
    'JF': 'Jazz Funk',
    'LJ': 'Latin Jazz',
    'LT': 'Latin',
    'MB': 'Military Band',
    'MS': 'Minstrelsy',
    'MT': 'Motown',
    'MU': 'Musical Theater',
    'NW': 'New Wave',
    'OP': 'Opera',
    'PO': 'Pop',
    'RB': 'Rhythm and Blues',
    'RO': 'Rock',
    'RT': 'Ragtime',
    'SA': 'Salsa',
    'SJ': 'Soul Jazz',
    'SM': 'Smooth Jazz',
    'SO': 'Soul',
    'ST': 'Stride Piano',
    'SW': 'Swing',
    'VV': 'Vaudeville',
    'WF': 'World Fusion'
}

## ensure the data reads as text
data['Genre'] = data['Genre'].astype(str)

## spell out the acronyms into a new column
def spell_out(codes):
    # Split codes by spaces and semicolons
    codes_list = codes.replace(';', ' ').split()
    return ' '.join([genre_mapping[code] for code in codes_list if code in genre_mapping])
data['All Genres'] = data['Genre'].apply(spell_out)

## print the number of unique combinations
genre_combinations = data['All Genres'].nunique()
print(f"Unique genre combinations: {genre_combinations}")

Unique genre combinations: 294


In [8]:
# Create new variables to represent each genre
for code in genre_mapping.keys():
    data[code] = data['Genre'].apply(lambda x: int(code in x))

# verify the results
genre_sums = {code: data[code].sum() for code in genre_mapping.keys()}
print("Total records for each genre:")
print(genre_sums)

Total records for each genre:
{'AC': 4, 'AF': 3, 'AJ': 33, 'AR': 9, 'BB': 2551, 'BG': 2, 'BL': 472, 'BN': 3, 'BR': 11, 'BS': 0, 'BW': 54, 'CH': 102, 'CI': 5, 'CL': 104, 'CV': 1, 'CY': 11, 'DI': 47, 'DW': 12, 'EJ': 1549, 'EL': 21, 'FJ': 1307, 'FK': 3, 'FO': 14, 'FU': 550, 'GJ': 699, 'GO': 229, 'HB': 1133, 'HH': 44, 'HO': 1, 'JB': 66, 'JF': 78, 'LJ': 5, 'LT': 2, 'MB': 1, 'MS': 21, 'MT': 1, 'MU': 7, 'NW': 11, 'OP': 54, 'PO': 291, 'RB': 743, 'RO': 165, 'RT': 79, 'SA': 7, 'SJ': 344, 'SM': 80, 'SO': 360, 'ST': 47, 'SW': 1858, 'VV': 25, 'WF': 3}


In [9]:
# rename the genre columns
data.rename(columns=genre_mapping, inplace=True)
data.columns

Index(['Arrival Country', 'Arrival State', 'Arrival County', 'Arrival City',
       'Musician Name', 'Arrival', 'Depart', 'Depart Country', 'Depart State',
       'Depart County', 'Depart City', 'Final Destination', 'Genre',
       'Instrument', 'Free', 'Roots', 'Dyn', 'Public', 'Dynasty Name',
       'MusicianID', 'SegmentID', 'All Genres', 'Afro-Cuban Jazz',
       'African Jazz', 'Acid Jazz', 'Acid Rock', 'Bebop', 'Boogaloo', 'Blues',
       'Bossa Nova', 'Broadway', 'Brass Band', 'Boogie-Woogie', 'Spirituals',
       'Circus', 'European Classical', 'Folk Cape Verde', 'Country', 'Disco',
       'Doo-Wop', 'Early Jazz', 'Easy Listening', 'Free Jazz', 'Funk',
       'Folk American', 'Fusion', 'General Jazz', 'Gospel', 'Hard Bop',
       'Hip Hop', 'House', 'Jump Blues', 'Jazz Funk', 'Latin Jazz', 'Latin',
       'Military Band', 'Minstrelsy', 'Motown', 'Musical Theater', 'New Wave',
       'Opera', 'Pop', 'Rhythm and Blues', 'Rock', 'Ragtime', 'Salsa',
       'Soul Jazz', 'Smooth Jazz

In [10]:
# sort and clean data
data = data.sort_values(by='SegmentID')
data

,Arrival Country,Arrival State,Arrival County,Arrival City,Musician Name,Arrival,Depart,Depart Country,Depart State,Depart County,...,Rock,Ragtime,Salsa,Soul Jazz,Smooth Jazz,Soul,Stride Piano,Swing,Vaudeville,World Fusion
118,Canada,New Brunswick,NaN,NaN,NaN,1817,1871.0,Canada,Nova Scotia,Cumberland,...,0,0,0,0,0,0,0,0,0,0
1764,United States,Georgia,Richmond,NaN,"Morgan, Lee",1829,1926.0,United States,Pennsylvania,Philadelphia,...,0,0,0,0,0,0,0,1,0,0
5877,United States,South Carolina,Greenwood,Callison,"Morgan, Lee",1870,1910.0,United States,South Carolina,Greenwood,...,0,0,0,0,0,0,0,1,0,0
5880,United States,South Carolina,Greenwood,Phoenix,"Morgan, Lee",1910,1920.0,United States,South Carolina,Greenwood,...,0,0,0,0,0,0,0,1,0,0
5878,United States,South Carolina,Greenwood,Greenwood,"Morgan, Lee",1920,1926.0,United States,Pennsylvania,Philadelphia,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7551,United States,Wisconsin,Milwaukee,Milwaukee,"Morgan, Frank",1911,1932.0,United States,Minnesota,Hennepin,...,0,0,0,0,0,0,0,0,0,0
2782,United States,Louisiana,Plaquemines,NaN,"Morgan, Isaiah",1836,1887.0,United States,Louisiana,Plaquemines,...,0,0,0,0,0,0,0,0,0,0
2789,United States,Louisiana,Plaquemines,Bertrandville,"Morgan, Isaiah",1887,1900.0,United States,Florida,Escambia,...,0,0,0,0,0,0,0,0,0,0
1205,United States,Florida,Escambia,Pensacola,"Morgan, Isaiah",1900,1913.0,United States,Louisiana,Plaquemines,...,0,0,0,0,0,0,0,0,0,0


In [11]:
# save as a CSV
data.to_csv(os.path.join(default_folder,"data_cleaned.csv"), index=False)
print(f"Saved to {default_folder}")

Saved to C:\Users\johnl\My Drive (jlauerma@pratt.edu)\Research\Music and Migration


### Mapping the data
The next step is to geocode the tabular data into `origins` points, `destinations` points, `paths` lines (which show an individual migration path), and `lifepaths` lines (which connect all of a musician's individual migration paths into one line). The workflow is designed to use the [ESRI World Geocoding Serivce](https://www.arcgis.com/home/item.html?id=305f2e55e67f4389bef269669fc2e284). 

In [12]:
# Load the CSV to geodatabase 
csv = os.path.join(default_folder, "data_cleaned.csv")
arcpy.conversion.TableToGeodatabase(csv, default_gdb)
print(f"loaded data to {default_gdb}")

loaded data to C:\Users\johnl\My Drive (jlauerma@pratt.edu)\Research\Music and Migration\MusicMigration.gdb


In [13]:
#geocode origins
arcpy.geocoding.GeocodeAddresses(
    in_table=os.path.join(default_gdb, "data_cleaned"),
    address_locator="https://geocode.arcgis.com/arcgis/rest/services/World/GeocodeServer/Esri World Geocoder",
    in_address_fields="'Address or Place' <None> VISIBLE NONE;Address2 <None> VISIBLE NONE;Address3 <None> VISIBLE NONE;Neighborhood <None> VISIBLE NONE;City Depart_City VISIBLE NONE;County Depart_County VISIBLE NONE;State Depart_State VISIBLE NONE;ZIP <None> VISIBLE NONE;ZIP4 <None> VISIBLE NONE;Country Depart_Country VISIBLE NONE",
    out_feature_class=os.path.join(default_gdb, "origins"),
    out_relationship_type="STATIC",
    country="AFG;ALB;DZA;ASM;AND;AGO;AIA;ATA;ATG;ARG;ARM;ABW;AUS;AUT;AZE;BHS;BHR;BGD;BRB;BLR;BEL;BLZ;BEN;BMU;BTN;BOL;BES;BIH;BWA;BVT;BRA;IOT;VGB;BRN;BGR;BFA;BDI;CPV;KHM;CMR;CAN;CYM;CAF;TCD;CHL;CHN;CXR;CCK;COL;COM;COK;CRI;HRV;CUB;CUW;CYP;CZE;CIV;COD;DNK;DJI;DMA;DOM;ECU;EGY;SLV;GNQ;ERI;EST;SWZ;ETH;EUR;FLK;FRO;FSM;FJI;FIN;FRA;GUF;PYF;ATF;GAB;GMB;GEO;DEU;GHA;GIB;GRC;GRL;GRD;GLP;GUM;GTM;GGY;GIN;GNB;GUY;HTI;HMD;HND;HKG;HUN;ISL;IND;IDN;IRN;IRQ;IRL;IMN;ISR;ITA;JAM;JPN;JEY;JOR;KAZ;KEN;KIR;KWT;KGZ;LAO;LVA;LBN;LSO;LBR;LBY;LIE;LTU;LUX;MAC;MDG;MWI;MYS;MDV;MLI;MLT;MHL;MTQ;MRT;MUS;MYT;MEX;MDA;MCO;MNG;MNE;MSR;MAR;MOZ;MMR;NAM;NRU;NPL;NLD;NCL;NZL;NIC;NER;NGA;NIU;NFK;PRK;MKD;MNP;NOR;OMN;PAK;PLW;PSE;PAN;PNG;PRY;PER;PHL;PCN;POL;PRT;PRI;QAT;COG;REU;ROU;RUS;RWA;BLM;SHN;KNA;LCA;MAF;SPM;VCT;WSM;SMR;STP;SAU;SEN;SRB;SYC;SLE;SGP;SXM;SVK;SVN;SLB;SOM;ZAF;SGS;KOR;SSD;ESP;LKA;SDN;SUR;SJM;SWE;CHE;SYR;TWN;TJK;TZA;THA;TLS;TGO;TKL;TON;TTO;TUN;TKM;TCA;TUV;TUR;VIR;UGA;UKR;ARE;GBR;USA;UMI;URY;UZB;VUT;VAT;VEN;VNM;WLF;ESH;YEM;ZMB;ZWE",
    location_type="ADDRESS_LOCATION",
    category=None,
    output_fields="LOCATION_ONLY"
)
print("geocoded the origin points")

geocoded the origin points


In [14]:
#geocode destinations
arcpy.geocoding.GeocodeAddresses(
    in_table=os.path.join(default_gdb, "data_cleaned"),
    address_locator="https://geocode.arcgis.com/arcgis/rest/services/World/GeocodeServer/Esri World Geocoder",
    in_address_fields="'Address or Place' <None> VISIBLE NONE;Address2 <None> VISIBLE NONE;Address3 <None> VISIBLE NONE;Neighborhood <None> VISIBLE NONE;City Arrival_City VISIBLE NONE;County Arrival_County VISIBLE NONE;State Arrival_State VISIBLE NONE;ZIP <None> VISIBLE NONE;ZIP4 <None> VISIBLE NONE;Country Arrival_Country VISIBLE NONE",
    out_feature_class=os.path.join(default_gdb, "destinations"),
    out_relationship_type="STATIC",
    country="AFG;ALB;DZA;ASM;AND;AGO;AIA;ATA;ATG;ARG;ARM;ABW;AUS;AUT;AZE;BHS;BHR;BGD;BRB;BLR;BEL;BLZ;BEN;BMU;BTN;BOL;BES;BIH;BWA;BVT;BRA;IOT;VGB;BRN;BGR;BFA;BDI;CPV;KHM;CMR;CAN;CYM;CAF;TCD;CHL;CHN;CXR;CCK;COL;COM;COK;CRI;HRV;CUB;CUW;CYP;CZE;CIV;COD;DNK;DJI;DMA;DOM;ECU;EGY;SLV;GNQ;ERI;EST;SWZ;ETH;EUR;FLK;FRO;FSM;FJI;FIN;FRA;GUF;PYF;ATF;GAB;GMB;GEO;DEU;GHA;GIB;GRC;GRL;GRD;GLP;GUM;GTM;GGY;GIN;GNB;GUY;HTI;HMD;HND;HKG;HUN;ISL;IND;IDN;IRN;IRQ;IRL;IMN;ISR;ITA;JAM;JPN;JEY;JOR;KAZ;KEN;KIR;KWT;KGZ;LAO;LVA;LBN;LSO;LBR;LBY;LIE;LTU;LUX;MAC;MDG;MWI;MYS;MDV;MLI;MLT;MHL;MTQ;MRT;MUS;MYT;MEX;MDA;MCO;MNG;MNE;MSR;MAR;MOZ;MMR;NAM;NRU;NPL;NLD;NCL;NZL;NIC;NER;NGA;NIU;NFK;PRK;MKD;MNP;NOR;OMN;PAK;PLW;PSE;PAN;PNG;PRY;PER;PHL;PCN;POL;PRT;PRI;QAT;COG;REU;ROU;RUS;RWA;BLM;SHN;KNA;LCA;MAF;SPM;VCT;WSM;SMR;STP;SAU;SEN;SRB;SYC;SLE;SGP;SXM;SVK;SVN;SLB;SOM;ZAF;SGS;KOR;SSD;ESP;LKA;SDN;SUR;SJM;SWE;CHE;SYR;TWN;TJK;TZA;THA;TLS;TGO;TKL;TON;TTO;TUN;TKM;TCA;TUV;TUR;VIR;UGA;UKR;ARE;GBR;USA;UMI;URY;UZB;VUT;VAT;VEN;VNM;WLF;ESH;YEM;ZMB;ZWE",
    location_type="ADDRESS_LOCATION",
    category=None,
    output_fields="LOCATION_ONLY"
)
print("geocoded the destination points")

geocoded the destination points


In [15]:
#merge the layers into one point layer
##define the new layers
origins_layer = os.path.join(default_gdb, "origins")
destinations_layer = os.path.join(default_gdb, "destinations")

## Add a LocationType field in origins
arcpy.management.AddField(origins_layer, "LocationType", "TEXT")
arcpy.management.CalculateField(origins_layer, "LocationType", '"Origin"', "PYTHON3")

##Calculate the LocationType field for origins and destinations
arcpy.management.AddField(destinations_layer, "LocationType", "TEXT")
arcpy.management.CalculateField(destinations_layer, "LocationType", '"Destination"', "PYTHON3")

##merge
arcpy.management.Merge(
    inputs="origins;destinations",
    output="origins_destinations_merged",
    field_mappings=None,
    add_source="NO_SOURCE_INFO",
    field_match_mode="AUTOMATIC"
)

print("created a point layer with both origins and destinations")

created a point layer with both origins and destinations


In [16]:
#create path segments between origins and destinations
with arcpy.EnvManager(qualifiedFieldNames="UNQUALIFIED"):
    arcpy.management.PointsToLine(
        Input_Features="origins_destinations_merged",
        Output_Feature_Class="paths",
        Line_Field="SegmentID",
        Sort_Field=None,
        Close_Line="NO_CLOSE",
        Line_Construction_Method="TWO_POINT",
        Attribute_Source="START",
        Transfer_Fields="OBJECTID;Arrival_Country;Arrival_State;Arrival_County;Arrival_City;Musician_Name;Arrival;Depart;Depart_Country;Depart_State;Depart_County;Depart_City;Final_Destination;Genre;Instrument;Free;Roots;Dyn;Public;Dynasty_Name;MusicianID;SegmentID;All_Genres;Afro_Cuban_Jazz;African_Jazz;Acid_Jazz;Acid_Rock;Bebop;Boogaloo;Blues;Bossa_Nova;Broadway;Brass_Band;Boogie_Woogie;Spirituals;Circus;European_Classical;Folk_Cape_Verde;Country;Disco;Doo_Wop;Early_Jazz;Easy_Listening;Free_Jazz;Funk;Folk_American;Fusion;General_Jazz;Gospel;Hard_Bop;Hip_Hop;House;Jump_Blues;Jazz_Funk;Latin_Jazz;Latin;Military_Band;Minstrelsy;Motown;Musical_Theater;New_Wave;Opera;Pop;Rhythm_and_Blues;Rock;Ragtime;Salsa;Soul_Jazz;Smooth_Jazz;Soul;Stride_Piano;Swing;Vaudeville;World_Fusion;LocationType"
    )
    
print("created a paths line layer")

created a paths line layer


In [17]:
# remove a naming convention that the PointsToLine tool adds...even though it has parameters that should stop it from doing that...

# List the field names
layer_name = "paths"
existing_fields = [field.name for field in arcpy.ListFields(layer_name)]

# Loop through fields. Change names and aliases.
for field in arcpy.ListFields(layer_name):
    if field.name.startswith("START_"):
        new_name = field.name.replace("START_", "")
        new_alias = field.aliasName.replace("START_", "") if field.aliasName.startswith("START_") else field.aliasName
        
        if new_name not in existing_fields:  # Avoid an error related to renaming existing fields
            arcpy.management.AlterField(layer_name, field.name, new_name, new_alias)
        else:
            print(f"Skipped {field.name} because {new_name} already exists.")

print("Field names and aliases updated.")

Skipped START_OBJECTID because OBJECTID already exists.
Skipped START_SegmentID because SegmentID already exists.
Field names and aliases updated.


In [18]:
#create path segements to follow life journeys of musicians
with arcpy.EnvManager(outputCoordinateSystem='GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]', qualifiedFieldNames="UNQUALIFIED"):
    arcpy.analysis.PairwiseDissolve(
        in_features=os.path.join(default_gdb, "paths"),
        out_feature_class=os.path.join(default_gdb, "life_paths"),
        dissolve_field="MusicianID",
        statistics_fields="Musician_Name FIRST;Final_Destination FIRST;Genre FIRST;All_Genres FIRST; Instrument FIRST;Free FIRST;Roots FIRST;Dyn FIRST;Dynasty_Name FIRST",
        multi_part="MULTI_PART",
        concatenation_separator=""
    ) 
print("created lifepaths line layer")

created lifepaths line layer


In [ ]:
# ditto for this line layer, to which ArcGIS appends "FIRST_" to every field
# remove a naming convention that the PointsToLine tool adds...even though it has parameters that should stop it from doing that...

# List the field names
layer_name = "life_paths"
existing_fields = [field.name for field in arcpy.ListFields(layer_name)]

# Loop through fields. Change names and aliases.
for field in arcpy.ListFields(layer_name):
    if field.name.startswith("FIRST_"):
        new_name = field.name.replace("FIRST_", "")
        new_alias = field.aliasName.replace("FIRST_", "") if field.aliasName.startswith("FIRST_") else field.aliasName
        
        if new_name not in existing_fields:  # Avoid an error related to renaming existing fields
            arcpy.management.AlterField(layer_name, field.name, new_name, new_alias)
        else:
            print(f"Skipped {field.name} because {new_name} already exists.")

print("Field names and aliases updated.")

In [20]:
#rank origins & destinations by path counts
##origins
arcpy.analysis.SpatialJoin(
    target_features="origins",
    join_features="paths",
    out_feature_class="origins_rankedbypaths",
    join_operation="JOIN_ONE_TO_ONE",
    join_type="KEEP_ALL",
    field_mapping='Arrival_Country "Arrival Country" true true false 8000 Text 0 0,First,#,origins,Arrival_Country,0,7999;Arrival_State "Arrival State" true true false 8000 Text 0 0,First,#,origins,Arrival_State,0,7999;Arrival_County "Arrival County" true true false 8000 Text 0 0,First,#,origins,Arrival_County,0,7999;Arrival_City "Arrival City" true true false 8000 Text 0 0,First,#,origins,Arrival_City,0,7999;Musician_Name "Musician Name" true true false 8000 Text 0 0,First,#,origins,Musician_Name,0,7999;Arrival "Arrival" true true false 4 Long 0 0,First,#,origins,Arrival,-1,-1;Depart "Depart" true true false 8 Double 0 0,First,#,origins,Depart,-1,-1;Depart_Country "Depart Country" true true false 8000 Text 0 0,First,#,origins,Depart_Country,0,7999;Depart_State "Depart State" true true false 8000 Text 0 0,First,#,origins,Depart_State,0,7999;Depart_County "Depart County" true true false 8000 Text 0 0,First,#,origins,Depart_County,0,7999;Depart_City "Depart City" true true false 8000 Text 0 0,First,#,origins,Depart_City,0,7999;Final_Destination "Final Destination" true true false 8000 Text 0 0,First,#,origins,Final_Destination,0,7999;Genre "Genre" true true false 8000 Text 0 0,First,#,origins,Genre,0,7999;Instrument "Instrument" true true false 8000 Text 0 0,First,#,origins,Instrument,0,7999;Free "Free" true true false 8000 Text 0 0,First,#,origins,Free,0,7999;Roots "Roots" true true false 8000 Text 0 0,First,#,origins,Roots,0,7999;Dyn "Dyn" true true false 8000 Text 0 0,First,#,origins,Dyn,0,7999;Public "Public" true true false 8000 Text 0 0,First,#,origins,Public,0,7999;Dynasty_Name "Dynasty Name" true true false 8000 Text 0 0,First,#,origins,Dynasty_Name,0,7999;MusicianID "MusicianID" true true false 4 Long 0 0,First,#,origins,MusicianID,-1,-1;SegmentID "SegmentID" true true false 8000 Text 0 0,First,#,origins,SegmentID,0,7999;All_Genres "All Genres" true true false 8000 Text 0 0,First,#,origins,All_Genres,0,7999;Afro_Cuban_Jazz "Afro-Cuban Jazz" true true false 4 Long 0 0,First,#,origins,Afro_Cuban_Jazz,-1,-1;African_Jazz "African Jazz" true true false 4 Long 0 0,First,#,origins,African_Jazz,-1,-1;Acid_Jazz "Acid Jazz" true true false 4 Long 0 0,First,#,origins,Acid_Jazz,-1,-1;Acid_Rock "Acid Rock" true true false 4 Long 0 0,First,#,origins,Acid_Rock,-1,-1;Bebop "Bebop" true true false 4 Long 0 0,First,#,origins,Bebop,-1,-1;Boogaloo "Boogaloo" true true false 4 Long 0 0,First,#,origins,Boogaloo,-1,-1;Blues "Blues" true true false 4 Long 0 0,First,#,origins,Blues,-1,-1;Bossa_Nova "Bossa Nova" true true false 4 Long 0 0,First,#,origins,Bossa_Nova,-1,-1;Broadway "Broadway" true true false 4 Long 0 0,First,#,origins,Broadway,-1,-1;Brass_Band "Brass Band" true true false 4 Long 0 0,First,#,origins,Brass_Band,-1,-1;Boogie_Woogie "Boogie-Woogie" true true false 4 Long 0 0,First,#,origins,Boogie_Woogie,-1,-1;Spirituals "Spirituals" true true false 4 Long 0 0,First,#,origins,Spirituals,-1,-1;Circus "Circus" true true false 4 Long 0 0,First,#,origins,Circus,-1,-1;European_Classical "European Classical" true true false 4 Long 0 0,First,#,origins,European_Classical,-1,-1;Folk_Cape_Verde "Folk Cape Verde" true true false 4 Long 0 0,First,#,origins,Folk_Cape_Verde,-1,-1;Country "Country" true true false 4 Long 0 0,First,#,origins,Country,-1,-1;Disco "Disco" true true false 4 Long 0 0,First,#,origins,Disco,-1,-1;Doo_Wop "Doo-Wop" true true false 4 Long 0 0,First,#,origins,Doo_Wop,-1,-1;Early_Jazz "Early Jazz" true true false 4 Long 0 0,First,#,origins,Early_Jazz,-1,-1;Easy_Listening "Easy Listening" true true false 4 Long 0 0,First,#,origins,Easy_Listening,-1,-1;Free_Jazz "Free Jazz" true true false 4 Long 0 0,First,#,origins,Free_Jazz,-1,-1;Funk "Funk" true true false 4 Long 0 0,First,#,origins,Funk,-1,-1;Folk_American "Folk American" true true false 4 Long 0 0,First,#,origins,Folk_American,-1,-1;Fusion "Fusion" true true false 4 Long 0 0,First,#,origins,Fusion,-1,-1;General_Jazz "General Jazz" true true false 4 Long 0 0,First,#,origins,General_Jazz,-1,-1;Gospel "Gospel" true true false 4 Long 0 0,First,#,origins,Gospel,-1,-1;Hard_Bop "Hard Bop" true true false 4 Long 0 0,First,#,origins,Hard_Bop,-1,-1;Hip_Hop "Hip Hop" true true false 4 Long 0 0,First,#,origins,Hip_Hop,-1,-1;House "House" true true false 4 Long 0 0,First,#,origins,House,-1,-1;Jump_Blues "Jump Blues" true true false 4 Long 0 0,First,#,origins,Jump_Blues,-1,-1;Jazz_Funk "Jazz Funk" true true false 4 Long 0 0,First,#,origins,Jazz_Funk,-1,-1;Latin_Jazz "Latin Jazz" true true false 4 Long 0 0,First,#,origins,Latin_Jazz,-1,-1;Latin "Latin" true true false 4 Long 0 0,First,#,origins,Latin,-1,-1;Military_Band "Military Band" true true false 4 Long 0 0,First,#,origins,Military_Band,-1,-1;Minstrelsy "Minstrelsy" true true false 4 Long 0 0,First,#,origins,Minstrelsy,-1,-1;Motown "Motown" true true false 4 Long 0 0,First,#,origins,Motown,-1,-1;Musical_Theater "Musical Theater" true true false 4 Long 0 0,First,#,origins,Musical_Theater,-1,-1;New_Wave "New Wave" true true false 4 Long 0 0,First,#,origins,New_Wave,-1,-1;Opera "Opera" true true false 4 Long 0 0,First,#,origins,Opera,-1,-1;Pop "Pop" true true false 4 Long 0 0,First,#,origins,Pop,-1,-1;Rhythm_and_Blues "Rhythm and Blues" true true false 4 Long 0 0,First,#,origins,Rhythm_and_Blues,-1,-1;Rock "Rock" true true false 4 Long 0 0,First,#,origins,Rock,-1,-1;Ragtime "Ragtime" true true false 4 Long 0 0,First,#,origins,Ragtime,-1,-1;Salsa "Salsa" true true false 4 Long 0 0,First,#,origins,Salsa,-1,-1;Soul_Jazz "Soul Jazz" true true false 4 Long 0 0,First,#,origins,Soul_Jazz,-1,-1;Smooth_Jazz "Smooth Jazz" true true false 4 Long 0 0,First,#,origins,Smooth_Jazz,-1,-1;Soul "Soul" true true false 4 Long 0 0,First,#,origins,Soul,-1,-1;Stride_Piano "Stride Piano" true true false 4 Long 0 0,First,#,origins,Stride_Piano,-1,-1;Swing "Swing" true true false 4 Long 0 0,First,#,origins,Swing,-1,-1;Vaudeville "Vaudeville" true true false 4 Long 0 0,First,#,origins,Vaudeville,-1,-1;World_Fusion "World Fusion" true true false 4 Long 0 0,First,#,origins,World_Fusion,-1,-1;LocationType "LocationType" true true false 255 Text 0 0,First,#,origins,LocationType,0,254;Shape_Length "Shape_Length" false true true 8 Double 0 0,First,#,paths,Shape_Length,-1,-1',
    match_option="WITHIN_A_DISTANCE",
    search_radius="5 Kilometers",
    distance_field_name="",
    match_fields=None
)
print("ranked the origins point layer")

##destinations
arcpy.analysis.SpatialJoin(
    target_features="destinations",
    join_features="paths",
    out_feature_class="destinations_rankedbypaths",
    join_operation="JOIN_ONE_TO_ONE",
    join_type="KEEP_ALL",
    field_mapping='Arrival_Country "Arrival Country" true true false 8000 Text 0 0,First,#,origins,Arrival_Country,0,7999;Arrival_State "Arrival State" true true false 8000 Text 0 0,First,#,origins,Arrival_State,0,7999;Arrival_County "Arrival County" true true false 8000 Text 0 0,First,#,origins,Arrival_County,0,7999;Arrival_City "Arrival City" true true false 8000 Text 0 0,First,#,origins,Arrival_City,0,7999;Musician_Name "Musician Name" true true false 8000 Text 0 0,First,#,origins,Musician_Name,0,7999;Arrival "Arrival" true true false 4 Long 0 0,First,#,origins,Arrival,-1,-1;Depart "Depart" true true false 8 Double 0 0,First,#,origins,Depart,-1,-1;Depart_Country "Depart Country" true true false 8000 Text 0 0,First,#,origins,Depart_Country,0,7999;Depart_State "Depart State" true true false 8000 Text 0 0,First,#,origins,Depart_State,0,7999;Depart_County "Depart County" true true false 8000 Text 0 0,First,#,origins,Depart_County,0,7999;Depart_City "Depart City" true true false 8000 Text 0 0,First,#,origins,Depart_City,0,7999;Final_Destination "Final Destination" true true false 8000 Text 0 0,First,#,origins,Final_Destination,0,7999;Genre "Genre" true true false 8000 Text 0 0,First,#,origins,Genre,0,7999;Instrument "Instrument" true true false 8000 Text 0 0,First,#,origins,Instrument,0,7999;Free "Free" true true false 8000 Text 0 0,First,#,origins,Free,0,7999;Roots "Roots" true true false 8000 Text 0 0,First,#,origins,Roots,0,7999;Dyn "Dyn" true true false 8000 Text 0 0,First,#,origins,Dyn,0,7999;Public "Public" true true false 8000 Text 0 0,First,#,origins,Public,0,7999;Dynasty_Name "Dynasty Name" true true false 8000 Text 0 0,First,#,origins,Dynasty_Name,0,7999;MusicianID "MusicianID" true true false 4 Long 0 0,First,#,origins,MusicianID,-1,-1;SegmentID "SegmentID" true true false 8000 Text 0 0,First,#,origins,SegmentID,0,7999;All_Genres "All Genres" true true false 8000 Text 0 0,First,#,origins,All_Genres,0,7999;Afro_Cuban_Jazz "Afro-Cuban Jazz" true true false 4 Long 0 0,First,#,origins,Afro_Cuban_Jazz,-1,-1;African_Jazz "African Jazz" true true false 4 Long 0 0,First,#,origins,African_Jazz,-1,-1;Acid_Jazz "Acid Jazz" true true false 4 Long 0 0,First,#,origins,Acid_Jazz,-1,-1;Acid_Rock "Acid Rock" true true false 4 Long 0 0,First,#,origins,Acid_Rock,-1,-1;Bebop "Bebop" true true false 4 Long 0 0,First,#,origins,Bebop,-1,-1;Boogaloo "Boogaloo" true true false 4 Long 0 0,First,#,origins,Boogaloo,-1,-1;Blues "Blues" true true false 4 Long 0 0,First,#,origins,Blues,-1,-1;Bossa_Nova "Bossa Nova" true true false 4 Long 0 0,First,#,origins,Bossa_Nova,-1,-1;Broadway "Broadway" true true false 4 Long 0 0,First,#,origins,Broadway,-1,-1;Brass_Band "Brass Band" true true false 4 Long 0 0,First,#,origins,Brass_Band,-1,-1;Boogie_Woogie "Boogie-Woogie" true true false 4 Long 0 0,First,#,origins,Boogie_Woogie,-1,-1;Spirituals "Spirituals" true true false 4 Long 0 0,First,#,origins,Spirituals,-1,-1;Circus "Circus" true true false 4 Long 0 0,First,#,origins,Circus,-1,-1;European_Classical "European Classical" true true false 4 Long 0 0,First,#,origins,European_Classical,-1,-1;Folk_Cape_Verde "Folk Cape Verde" true true false 4 Long 0 0,First,#,origins,Folk_Cape_Verde,-1,-1;Country "Country" true true false 4 Long 0 0,First,#,origins,Country,-1,-1;Disco "Disco" true true false 4 Long 0 0,First,#,origins,Disco,-1,-1;Doo_Wop "Doo-Wop" true true false 4 Long 0 0,First,#,origins,Doo_Wop,-1,-1;Early_Jazz "Early Jazz" true true false 4 Long 0 0,First,#,origins,Early_Jazz,-1,-1;Easy_Listening "Easy Listening" true true false 4 Long 0 0,First,#,origins,Easy_Listening,-1,-1;Free_Jazz "Free Jazz" true true false 4 Long 0 0,First,#,origins,Free_Jazz,-1,-1;Funk "Funk" true true false 4 Long 0 0,First,#,origins,Funk,-1,-1;Folk_American "Folk American" true true false 4 Long 0 0,First,#,origins,Folk_American,-1,-1;Fusion "Fusion" true true false 4 Long 0 0,First,#,origins,Fusion,-1,-1;General_Jazz "General Jazz" true true false 4 Long 0 0,First,#,origins,General_Jazz,-1,-1;Gospel "Gospel" true true false 4 Long 0 0,First,#,origins,Gospel,-1,-1;Hard_Bop "Hard Bop" true true false 4 Long 0 0,First,#,origins,Hard_Bop,-1,-1;Hip_Hop "Hip Hop" true true false 4 Long 0 0,First,#,origins,Hip_Hop,-1,-1;House "House" true true false 4 Long 0 0,First,#,origins,House,-1,-1;Jump_Blues "Jump Blues" true true false 4 Long 0 0,First,#,origins,Jump_Blues,-1,-1;Jazz_Funk "Jazz Funk" true true false 4 Long 0 0,First,#,origins,Jazz_Funk,-1,-1;Latin_Jazz "Latin Jazz" true true false 4 Long 0 0,First,#,origins,Latin_Jazz,-1,-1;Latin "Latin" true true false 4 Long 0 0,First,#,origins,Latin,-1,-1;Military_Band "Military Band" true true false 4 Long 0 0,First,#,origins,Military_Band,-1,-1;Minstrelsy "Minstrelsy" true true false 4 Long 0 0,First,#,origins,Minstrelsy,-1,-1;Motown "Motown" true true false 4 Long 0 0,First,#,origins,Motown,-1,-1;Musical_Theater "Musical Theater" true true false 4 Long 0 0,First,#,origins,Musical_Theater,-1,-1;New_Wave "New Wave" true true false 4 Long 0 0,First,#,origins,New_Wave,-1,-1;Opera "Opera" true true false 4 Long 0 0,First,#,origins,Opera,-1,-1;Pop "Pop" true true false 4 Long 0 0,First,#,origins,Pop,-1,-1;Rhythm_and_Blues "Rhythm and Blues" true true false 4 Long 0 0,First,#,origins,Rhythm_and_Blues,-1,-1;Rock "Rock" true true false 4 Long 0 0,First,#,origins,Rock,-1,-1;Ragtime "Ragtime" true true false 4 Long 0 0,First,#,origins,Ragtime,-1,-1;Salsa "Salsa" true true false 4 Long 0 0,First,#,origins,Salsa,-1,-1;Soul_Jazz "Soul Jazz" true true false 4 Long 0 0,First,#,origins,Soul_Jazz,-1,-1;Smooth_Jazz "Smooth Jazz" true true false 4 Long 0 0,First,#,origins,Smooth_Jazz,-1,-1;Soul "Soul" true true false 4 Long 0 0,First,#,origins,Soul,-1,-1;Stride_Piano "Stride Piano" true true false 4 Long 0 0,First,#,origins,Stride_Piano,-1,-1;Swing "Swing" true true false 4 Long 0 0,First,#,origins,Swing,-1,-1;Vaudeville "Vaudeville" true true false 4 Long 0 0,First,#,origins,Vaudeville,-1,-1;World_Fusion "World Fusion" true true false 4 Long 0 0,First,#,origins,World_Fusion,-1,-1;LocationType "LocationType" true true false 255 Text 0 0,First,#,origins,LocationType,0,254;Shape_Length "Shape_Length" false true true 8 Double 0 0,First,#,paths,Shape_Length,-1,-1',
    match_option="WITHIN_A_DISTANCE",
    search_radius="5 Kilometers",
    distance_field_name="",
    match_fields=None
)
print("ranked the destinations point layer")

ranked the origins point layer
ranked the destinations point layer
